This demo uses the implemented story RSPY-120.

ASSUMPTION: All the files from CADIP and ADGS mockup stations are already ingested and well published in the catalog. The ingestion of these files can be performed by running the demo prefect_flows_with_catalog_support from sprint_7 prior running this demo

## Configuration

In [ ]:
import getpass
import os

# Get the RS-Server website URL. 
# Note: this is None in local mode, we rather use the env vars RSPY_HOST_CADIP, RSPY_HOST_CATALOG, ...
url = os.getenv("RSPY_WEBSITE")

# Local mode or hybrid/cluster
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")
 
# Manually enter S3 access and secret key.
# This is needed only for manual operation in this demo. This is not needed by rs-client.
if not os.getenv("S3_ACCESSKEY"):
    os.environ["S3_ACCESSKEY"] = getpass.getpass(f"Enter S3 access key for {os.environ['S3_ENDPOINT']!r}:")
if not os.getenv("S3_SECRETKEY"):
    os.environ["S3_SECRETKEY"] = getpass.getpass(f"Enter S3 secret key for {os.environ['S3_ENDPOINT']!r}:")

# In hybrid/cluster mode: read the API key
apikey = os.getenv("RSPY_APIKEY")
HEADERS={}
if local_mode:
    url_dpr_sim = "http://dpr-simulator:8000"
else:
    url_dpr_sim = "http://mockup-processor-dpr-svc.processing.svc.cluster.local:8080"
    if not apikey:
        apikey = getpass.getpass(f"Enter your API key from {url!r}:")
    HEADERS={"headers": {"x-api-key": apikey}}

Install the needed libraries

In [ ]:
!pip install boto3

In [ ]:
# Instrument these notebook cells with OpenTelemetry (optional)
from opentelemetry import trace
otel_tracer = trace.get_tracer("my.notebook.trace")

In [ ]:
# Check if there are the same products as this demo will process in the final catalog bucket

import boto3
import os

if local_mode:
    delete_files = True
else:
    delete_files = input("Before running the demo, check if previous results are in the final bucket. If these are to be found, should they be removed (optional)? (y/n): ").lower().strip() == 'y'

s3_dir = "PRODUCTS_DEMO_SPRINT_8"

s3_session = boto3.session.Session()
s3_client = s3_session.client(
    service_name="s3",
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)

bucket_name = "rs-cluster-catalog"
none_found = True
for bucket in s3_client.list_buckets()["Buckets"]:
    if bucket_name != bucket["Name"]:
        continue
    if 'Contents' in s3_client.list_objects(Bucket=bucket_name):
        objects = s3_client.list_objects(Bucket=bucket_name)['Contents']
        for obj in objects:
            if s3_dir in obj["Key"]:                
                if delete_files:
                    print(f"Deleting {obj['Key']}")
                    s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
                else:                          
                    print(f"{obj['Key']} modified on {obj['LastModified']}") 
                    none_found = False
print("No products found in the catalog bucket") if none_found else print("\nProducts found in the catalog bucket")

In [ ]:
from rs_client.rs_client import RsClient
from rs_common.config import ECadipStation, EPlatform

# Define the owner/user ID to read/write catalog collections.
# Your API key must give you the read/write permissions for this owner ID.
owner_id = "DemoUser"

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key
#   - Owner ID
#   - Logger (optional, a default one can be used)
generic_client = RsClient(url, apikey, owner_id, logger=None)

# From this generic instance, get an Auxip client instance
# auxip_client = generic_client.get_auxip_client() # not used in this demo

# Or get a Cadip client instance. Pass the cadip station and the platforms.
cadip_station = ECadipStation.CADIP
cadip_client = generic_client.get_cadip_client(cadip_station)

# Or get a Stac client instance
stac_client = generic_client.get_stac_client()

In [ ]:
import requests

# Clean previous executions
requests.delete(f"{stac_client.href_catalog}/catalog/collections/DemoUser:s1_dpr", **HEADERS)

In [ ]:
from datetime import datetime

# Search cadip sessions by date interval
start_date = datetime(2014, 1, 1, 12, 0, 0)
stop_date = datetime(2024, 1, 1, 12, 0, 0)
platforms = [EPlatform.S1A, EPlatform.S2B]
sessions = cadip_client.search_sessions(timeout=30, start_date=start_date, stop_date=stop_date, platforms=platforms)

# Keep the first one
session_id = sessions[0]["id"]
print (session_id)

In [ ]:
# Assumption: all the files from CADIP and ADGS server are well ingested in the catalog  as well as in the buckets
# Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given in the docker-compose.yml file.
with otel_tracer.start_as_current_span("nb.run-flow-s1_l0") as span:
    
    from rs_workflows.s1_l0 import (
        s1_l0_flow,
        PrefectS1L0FlowConfig,
    )
    
    mission = "s1"
    s3_storage = f"s3://rs-cluster-final/{s3_dir}/"
    temp_s3_storage = f"s3://rs-cluster-temp/{s3_dir}/"
    
    # the type of the prducts that the DPR should process
    product_types = ["S1SEWRAW", "S1SIWRAW", "S1SSMRAW", "S1SWVRAW"]

    # gather the data from ADGS. Excerpt from the RSPY-120 user story:
    # "3. search in the STAC collection rs-ops/s1_aux for the three required AUX
    # items (given that RSPY-115 has filled the collection). As we don't know yet the
    # rules that will be specified by DPR to retrieve the correct AUX data, we will for
    # now hardcode the AUX file names as below:
    adgs_files = [
        "S1A_AUX_PP2_V20200106T080000_G20200106T080000.SAFE",
        "S1A_OPER_MPL_ORBPRE_20200409T021411_20200416T021411_0001.EOF",
        "S1A_OPER_AUX_RESORB_OPOD_20210716T110702_V20210716T071044_20210716T102814.EOF",
    ]
    
    s1_l0_flow(
            PrefectS1L0FlowConfig(
                stac_client,
                url_dpr_sim,
                mission,
                session_id,
                product_types,
                adgs_files,
                s3_storage,
                temp_s3_storage,
            ),
        )